In [1]:
#Documento creado por https://github.com/Stepp1
#Modificado por esteban

import subprocess
import shlex
import requests

In [2]:
#!pip install tabula-py 
#Sólo una vez

     |████████████████████████████████| 10.4 MB 787 kB/s eta 0:00:01    |████████████████▊               | 5.4 MB 734 kB/s eta 0:00:07     |█████████████████               | 5.6 MB 734 kB/s eta 0:00:07


### Eliminamos descargas anteriores

In [0]:
#!rm *pdf

### Obtenemos las urls de los informes (EPI y Generales)

In [4]:
response = subprocess.check_output(shlex.split('curl --request GET https://www.gob.cl/coronavirus/cifrasoficiales/'))


url_reporte = []
url_informe_epi = []
for line in response.decode().splitlines():
    if "Reporte_Covid19.pdf" in line:
        url = line.strip().split('https://')[1].split("\"")[0]
        
        url_reporte.append(url)
        
    elif "INFORME_EPI" in line:
        
        test = line.strip()
        test = test.split('https://')[1].split("\"")[0]
        url_informe_epi.append(test)
        
        

In [5]:
response


b'<!DOCTYPE html>\n<html lang="es">\n  <head>\n    <meta charset="utf-8">\n    <title>Gob.cl - Cifras Oficiales</title>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="author" content="https://magnet.cl">\n    <meta name="twitter:site" content="@Gobiernodechile">\n    <meta name="twitter:creator" content="@Gobiernodechile">\n    <meta name="og:site_name" content="Gobierno de Chile">\n    <meta name="og:type" content="website">\n    <meta name="fb:admins" content="572738868">\n    <meta name="fb:app_id" content="114405611929952">\n    <meta itemprop="description" content="">\n    <meta itemprop="image" content="http://www.gob.clhttps://cdn.digital.gob.cl/favicon/ms-icon-310x310.png">\n    <meta itemprop="name" content="Gob.cl: ">\n    <meta name="keywords" content="">\n    <meta name="description" content="">\n    <link href="//maxcdn.bootstrapcdn.com/font-awesome/4.2.0/css/font-awesome.min.css" rel="stylesheet">\n    <link rel="stylesheet" h

#### Double Check

In [0]:
url_reporte

['cdn.digital.gob.cl/public_files/Campa%C3%B1as/Corona-Virus/Reportes/02.04.2020_Reporte_Covid19.pdf',
 'cdn.digital.gob.cl/public_files/Campa%C3%B1as/Corona-Virus/Reportes/05.04.2020_Reporte_Covid19.pdf',
 'cdn.digital.gob.cl/public_files/Campa%C3%B1as/Corona-Virus/Reportes/04.04.2020_Reporte_Covid19.pdf',
 'cdn.digital.gob.cl/public_files/Campa%C3%B1as/Corona-Virus/Reportes/03.04.2020_Reporte_Covid19.pdf',
 'cdn.digital.gob.cl/public_files/Campa%C3%B1as/Corona-Virus/Reportes/02.04.2020_Reporte_Covid19.pdf',
 'cdn.digital.gob.cl/public_files/Campa%C3%B1as/Corona-Virus/Reportes/01.04.2020_Reporte_Covid19.pdf']

In [0]:
url_informe_epi

['cdn.digital.gob.cl/public_files/Campa%C3%B1as/Corona-Virus/Reportes/INFORME_EPI_COVID19_20200401v2.pdf',
 'cdn.digital.gob.cl/public_files/Campa%C3%B1as/Corona-Virus/Reportes/INFORME_EPI_COVID19_20200330.pdf']

# Descarga Informes

In [0]:
for url in set(url_reporte):
    subprocess.check_output(shlex.split("wget "+ url))

In [0]:
for url in set(url_informe_epi):
    subprocess.check_output(shlex.split("wget "+ url))

In [0]:
!ls

01.04.2020_Reporte_Covid19.pdf	05.04.2020_Reporte_Covid19.pdf
02.04.2020_Reporte_Covid19.pdf	INFORME_EPI_COVID19_20200330.pdf
03.04.2020_Reporte_Covid19.pdf	INFORME_EPI_COVID19_20200401v2.pdf
04.04.2020_Reporte_Covid19.pdf	sample_data


# Preprocesamiento

Usamos tabula-py: wrapper de Tabula App (escrita en Java). A library for extracting tables from PDF files 
https://github.com/chezou/tabula-py

In [0]:
import tabula

dfs_files = {}
for url in url_informe_epi:
    pdf_file = url.split('/')[-1]
    df = tabula.read_pdf(pdf_file, pages='all', multiple_tables=True)
    
    fecha = pdf_file.split('_')[-1].split('.')[0]
    print(fecha)
    dfs_files['tablas_' + fecha] = df
              
              

20200401v2


Got stderr: Apr 05, 2020 7:59:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 05, 2020 7:59:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 05, 2020 7:59:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



20200330


Verificamos algunas tablas

In [0]:
tablas_20200401 = dfs_files['tablas_20200401v2']
tablas_20200330 = dfs_files['tablas_20200330']

In [0]:
df_comunas_20200401 = {}
unnamed_primeraCol = {}
for idx, df in enumerate(tablas_20200401):

    if 'Comuna' in df.columns:
        
        key= 'tabla_' + str(idx + 1)
        print(key)
        df_comunas_20200401[key] = df

tabla_6
tabla_7
tabla_8
tabla_9
tabla_10
tabla_11
tabla_14
tabla_15
tabla_16
tabla_17
tabla_18
tabla_19
tabla_20
tabla_21
tabla_22


In [0]:
df_comunas_20200401['tabla_6'].head()

,Unnamed: 0,Comuna,N°,Unnamed: 1,Población,Tasa,Unnamed: 2
0,Arica,NaN,NaN,6,247552,NaN,"2,4"
1,Camarones,NaN,NaN,-,1233,NaN,0
2,General Lagos,NaN,NaN,-,810,NaN,0
3,Putre,NaN,NaN,-,2515,NaN,0
4,Total,NaN,NaN,6,252110,NaN,"2,4"


Tabla empieza con un *Unnamed: 0*

In [0]:
df_comunas_20200401['tabla_22'].head()

,Comuna,N°,Unnamed: 0,Población,Unnamed: 1,Tasa,Unnamed: 2
0,Antártica,NaN,-,137,NaN,NaN,"0,0"
1,Cabo de Hornos,NaN,6,1983,NaN,NaN,"302,6"
2,Laguna Blanca,NaN,-,264,NaN,NaN,"0,0"
3,Natales,NaN,4,23782,NaN,NaN,"16,8"
4,Porvenir,NaN,-,7323,NaN,NaN,"27,3"


Tabla **no** empieza con un *Unnamed: 0*

In [0]:
df_comunas_20200330 = {}
unnamed_primeraCol = {}
for idx, df in enumerate(tablas_20200330):

    if 'Comuna' in df.columns:
        
        key= 'tabla_' + str(idx + 1)
        print(key)
        df_comunas_20200330[key] = df

tabla_7
tabla_8
tabla_9
tabla_10
tabla_11
tabla_12
tabla_13
tabla_15
tabla_16
tabla_17
tabla_18
tabla_19
tabla_20
tabla_21
tabla_22
tabla_23


In [0]:
df_comunas_20200330['tabla_7'].head()

,Unnamed: 0,Comuna,N°,Unnamed: 1,Población,Tasa,Unnamed: 2
0,Arica,NaN,NaN,6,247552,NaN,"2,4"
1,Camarones,NaN,NaN,-,1233,NaN,"0,0"
2,General Lagos,NaN,NaN,-,810,NaN,"0,0"
3,Putre,NaN,NaN,-,2515,NaN,"0,0"
4,Total,NaN,NaN,6,252110,NaN,"2,4"


Misma tabla empieza con un *Unnamed: 0*

In [0]:
df_comunas_20200330['tabla_23'].head()

,Comuna,N°,Unnamed: 0,Población,Unnamed: 1,Tasa,Unnamed: 2
0,Antártica,NaN,-,137,NaN,NaN,"0,0"
1,Cabo de Hornos,NaN,5,1983,NaN,NaN,"252,1"
2,Laguna Blanca,NaN,-,264,NaN,NaN,"0,0"
3,Natales,NaN,-,23782,NaN,NaN,"4,2"
4,Porvenir,NaN,-,7323,NaN,NaN,"0,0"


Misma tabla **no** empieza con un *Unnamed: 0*

# Separamos estas dos categorias:

In [0]:
df_comunas_20200401 = {}
unnamed_primeraCol_20200401 = {}

for idx, df in enumerate(tablas_20200401):
    if 'Comuna' in df.columns:
        
        key = 'tabla_' + str(idx + 1)
        df_comunas_20200401[key] = df
        
        if 'Unnamed' in df.columns[0]: 
            print(key)
            unnamed_primeraCol_20200401[key] = df

tabla_6
tabla_17
tabla_18
tabla_21


In [0]:
df_comunas_20200330 = {}
unnamed_primeraCol_20200330 = {}

for idx, df in enumerate(tablas_20200330):
    if 'Comuna' in df.columns:
        
        key= 'tabla_' + str(idx + 1)
        df_comunas_20200330[key] = df
        
        if 'Unnamed' in df.columns[0]:
            print(key)            
            unnamed_primeraCol_20200330[key] = df

tabla_7
tabla_13
tabla_18
tabla_19
tabla_22


# Resumen

* El informe 20200330 tiene una tabla más al parecer (en realidad esto no es así y parecer que un cambio en el gráfico dejo la kgá).

* La extracción de tablas parece tener los mismos errores en las mismas tablas.

In [0]:
%%capture
"""

for tup_1, tup_2 in zip(df_comunas.items(), df_comunas_2.items()):
    key_1, df_1 = tup_1
    key_2, df_2 = tup_2
    
    if (key_1 or key_2) in unnamed_primeraCol:
        if (df_1.columns == df_2.columns).all: 
            print("LAS COLUMNAS DE LAS TABLAS *diferentes* coinciden!", key_1, key_2)
            
"""

## Estandarizamos las tablas

In [0]:
for key in df_comunas_20200401.keys():
    df = df_comunas_20200401[key]
    
    if key in unnamed_primeraCol_20200401.keys():
        df['Comuna'] = df['Unnamed: 0']
        df['N°'] = df['Unnamed: 1']
        df['Tasa'] = df['Unnamed: 2']
        
        df_comunas_20200401[key] = df.drop(labels='Unnamed: 0', 
                                  axis=1).drop(labels='Unnamed: 1',
                                               axis=1).drop(labels='Unnamed: 2', axis=1)
        
    else:
        if key == 'tabla_22': continue  
        
        df_comunas_20200401[key] = df.drop(labels='Unnamed: 0', 
                          axis=1).drop(labels='Unnamed: 1', axis=1)

In [0]:
for key in df_comunas_20200330.keys():
    df = df_comunas_20200330[key]
    
    if key in unnamed_primeraCol_20200330.keys():
        df['Comuna'] = df['Unnamed: 0']
        df['N°'] = df['Unnamed: 1']
        df['Tasa'] = df['Unnamed: 2']
        
        df_comunas_20200330[key] = df.drop(labels='Unnamed: 0', 
                                  axis=1).drop(labels='Unnamed: 1',
                                               axis=1).drop(labels='Unnamed: 2', axis=1)
    else:
        if key == 'tabla_22': continue  
        
        df_comunas_20200330[key] = df.drop(labels='Unnamed: 0', 
                          axis=1).drop(labels='Unnamed: 1',axis=1)

In [0]:
for key, region in df_comunas_20200401.items():
    print(key, region.columns)

tabla_6 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_7 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_8 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_9 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_10 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_11 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_14 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_15 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_16 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_17 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_18 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_19 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_20 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_21 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_22 Index(['Comuna'

In [0]:
for key, region in df_comunas_20200330.items():
    print(key, region.columns)

tabla_7 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_8 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_9 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_10 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_11 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_12 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_13 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_15 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_16 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_17 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_18 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_19 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_20 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_21 Index(['Comuna', 'N°', 'Población', 'Tasa'], dtype='object')
tabla_22 Index(['Comuna

## Ultima tabla tiene *Unnamed: 2*

In [0]:
df_comunas_20200401['tabla_21']

,Comuna,N°,Población,Tasa
0,Aysén,-,25002,"4,0"
1,Chile Chico,-,5121,"0,0"
2,Cisnes,-,5828,"0,0"
3,Cochrane,-,3685,"0,0"
4,Coyhaique,-,61210,"3,3"
5,Guaitecas,-,1599,"0,0"
6,Lago Verde,-,920,"0,0"
7,O'Higgins,-,661,"0,0"
8,Río Ibáñez,-,2699,"0,0"
9,Tortel,-,572,"0,0"


In [0]:
df_comunas_20200330['tabla_23']

,Comuna,N°,Población,Tasa,Unnamed: 2
0,Antártica,NaN,137,NaN,"0,0"
1,Cabo de Hornos,NaN,1983,NaN,"252,1"
2,Laguna Blanca,NaN,264,NaN,"0,0"
3,Natales,NaN,23782,NaN,"4,2"
4,Porvenir,NaN,7323,NaN,"0,0"
5,Primavera,NaN,694,NaN,"0,0"
6,Punta Arenas,NaN,141984,NaN,"20,4"
7,Río Verde,NaN,211,NaN,"0,0"
8,San Gregorio,NaN,681,NaN,"0,0"
9,Timaukel,NaN,282,NaN,"0,0"
